In [1]:
import pandas as pd
from nba_api.stats.endpoints import leaguedashplayerstats
# URL of the local HTML file being served
df=pd.read_csv('lebron_link25.csv')
df['year']=2025
df
df.dropna(subset=['Name'],inplace=True)
old_df=pd.read_csv('lebron.csv')
old_df=old_df[old_df.year<2025]
old_df
column_mapping = {
    'Season': 'Season',
    'Name': 'Player',
    'Team': 'team',
    'Minutes': 'Minutes',
    'Position': 'Pos',
    'offensiveArchetype': 'Offensive Archetype',
    'LEBRON_WAR': 'WAR',
    'LEBRON': 'LEBRON',
    'OLEBRON': 'O-LEBRON',
    'DLEBRON': 'D-LEBRON',
    'defensive_role': 'Defensive Role',
    'PLAYER_ID': 'NBA ID',
    'G': 'Games',
    # 'bref_id' and 'Age' omitted intentionally
}
df.rename(columns=column_mapping,inplace=True)

samecolumns =set(old_df.columns) - (set(old_df.columns)-set(df.columns))
samecolumns
df=df[list(samecolumns)]
print(old_df.columns)

def update(year):
    season_string = str(year-1) +'-'+str(year)[-2::]
    print(season_string)
    frames = leaguedashplayerstats.LeagueDashPlayerStats(season=season_string).get_data_frames()
    df = frames[0]

    return dict(zip(df['PLAYER_ID'],df['AGE']))
ages = update(2025)
df['Age']=df['NBA ID'].map(ages)
idframe=pd.read_csv('index_master.csv')
id_map = dict(zip(idframe['nba_id'],idframe['bref_id']))

df['bref_id']=df['NBA ID'].map(id_map)
df['Player']=df['Player'].str.lower()

newframe = pd.concat([old_df,df])
newframe.to_csv('lebron.csv',index=False)
newframe.to_csv('../web_app/data/lebron.csv',index=False)

newframe

Index(['Season', 'Player', 'team', 'Minutes', 'Pos', 'Offensive Archetype',
       'WAR', 'LEBRON', 'O-LEBRON', 'D-LEBRON', 'year', 'Defensive Role',
       'NBA ID', 'bref_id', 'Games', 'Age'],
      dtype='object')
2024-25


,Season,Player,team,Minutes,Pos,Offensive Archetype,WAR,LEBRON,O-LEBRON,D-LEBRON,year,Defensive Role,NBA ID,bref_id,Games,Age
0,2009-10,lebron james,CLE,2966.000000,SF,Shot Creator,20.570000,8.840000,7.310000,1.530000,2010,NaN,2544.0,jamesle01,76,25.0
1,2009-10,rodrigue beaubois,DAL,700.000000,PG,Primary Ball Handler,0.640000,-0.930000,0.090000,-1.030000,2010,NaN,201958.0,beaubro01,56,21.0
2,2009-10,david andersen,HOU,891.000000,C,Post Scorer,0.800000,-0.950000,-0.970000,0.020000,2010,NaN,2431.0,anderda03,63,29.0
3,2009-10,eric gordon,LAC,2229.000000,SG,Secondary Ball Handler,2.010000,-0.950000,1.140000,-2.090000,2010,NaN,201569.0,gordoer01,62,21.0
4,2009-10,michael beasley,MIA,2328.000000,PF,Athletic Finisher,2.030000,-0.990000,-0.990000,0.000000,2010,NaN,201563.0,beaslmi01,78,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,2024-25,josh green,CHO,381.968333,SG,Stationary Shooter,-0.179606,-3.189825,-3.248599,0.058773,2025,Point of Attack,1630182.0,greenjo02,15,24.0
481,2024-25,bilal coulibaly,WAS,493.323333,SF,Secondary Ball Handler,-0.247193,-3.299425,-2.185336,-1.114090,2025,Point of Attack,1641731.0,coulibi01,14,20.0
482,2024-25,brandon ingram,NOP,539.465000,SF,Shot Creator,-0.291969,-3.305432,-0.383406,-2.922026,2025,Helper,1627742.0,ingrabr01,16,27.0
483,2024-25,corey kispert,WAS,438.628333,SF,Off Screen Shooter,-0.302467,-3.632959,-1.448381,-2.184579,2025,Low Activity,1630557.0,kispeco01,15,25.0
